In [17]:
# 3-dimensional embeddings of the sequence “Your journey starts with one step”

import torch
inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your (x^1)
   [0.55, 0.87, 0.66], # journey (x^2)
   [0.57, 0.85, 0.64], # starts (x^3)
   [0.22, 0.58, 0.33], # with (x^4)   
   [0.77, 0.25, 0.10], # one (x^5)   
   [0.05, 0.80, 0.55]] # step (x^6)   
)

# attention scores for 2nd input ("journey") wrt to all other inputs - using dot products
# mutliplies element wise i.e. 1st ele with 1st ele, 2nd with 2nd and so on
query = inputs[1]                           
attn_scores_2 = torch.empty(inputs.shape[0])
for i, x_i in enumerate(inputs):
    attn_scores_2[i] = torch.dot(x_i, query)
print(attn_scores_2)

# normalize the attention scores to convert them to attention weights
attn_weights_2 = torch.softmax(attn_scores_2, dim=0)
print("Attention weights:", attn_weights_2)
print("Sum:", attn_weights_2.sum())

# computing z(2) by multiplying the embedded input tokens, x(i), with the corresponding attention weights and then summing the resulting vectors
query = inputs[1]        
context_vec_2 = torch.zeros(query.shape)
for i,x_i in enumerate(inputs):
    context_vec_2 += attn_weights_2[i]*x_i
print(context_vec_2)



tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])
Attention weights: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum: tensor(1.)
tensor([0.4419, 0.6515, 0.5683])


In [18]:
attn_scores = inputs @ inputs.T
attn_weights = torch.softmax(attn_scores, dim=-1)

all_context_vecs = attn_weights @ inputs
print(all_context_vecs)

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])


In [19]:
# now with trainable weights
# we compute 3 matrices: query Wq (for current input), key Wk, value Wv
# taking 2nd input as query
x_2 = inputs[1]    
d_in = inputs.shape[1]     
d_out = 2
# Note that in GPT-like models, the input and output dimensions are usually the same, but to better follow the computation, we’ll use different input (d_in=3) and output (d_out=2) dimensions here.

W_query = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False) 
W_key   = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False) 
W_value = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)

query_2 = x_2 @ W_query
keys = inputs @ W_key 
values = inputs @ W_value

print(d_in, x_2, W_query.shape, W_query)
print(query_2)
print(keys)


3 tensor([0.5500, 0.8700, 0.6600]) torch.Size([3, 2]) Parameter containing:
tensor([[0.7662, 0.8018],
        [0.6371, 0.3464],
        [0.2020, 0.1266]])
tensor([1.1090, 0.8259])
tensor([[0.8591, 0.4354],
        [1.0358, 0.7199],
        [1.0238, 0.7259],
        [0.5483, 0.3659],
        [0.5193, 0.6296],
        [0.7036, 0.3352]])


In [20]:
attn_scores_2 = query_2 @ keys.T      
print(attn_scores_2)

d_k = keys.shape[-1] 
attn_weights_2 = torch.softmax(attn_scores_2 / d_k**0.5, dim=-1) 
print(attn_weights_2)

context_vec_2 = attn_weights_2 @ values 
print(context_vec_2)

tensor([1.3124, 1.7432, 1.7349, 0.9103, 1.0959, 1.0571])
tensor([0.1626, 0.2205, 0.2192, 0.1224, 0.1395, 0.1358])
tensor([1.0427, 0.5773])


In [21]:
import torch.nn as nn 
class SelfAttention_v1(nn.Module):    
    def __init__(self, d_in, d_out): 
        super().__init__() 
        self.W_query = nn.Parameter(torch.rand(d_in, d_out)) 
        self.W_key   = nn.Parameter(torch.rand(d_in, d_out)) 
        self.W_value = nn.Parameter(torch.rand(d_in, d_out))

    def forward(self, x): 
        keys = x @ self.W_key 
        queries = x @ self.W_query 
        values = x @ self.W_value 
        attn_scores = queries @ keys.T # omega 
        attn_weights = torch.softmax( attn_scores / keys.shape[-1]**0.5, dim=-1 ) 
        context_vec = attn_weights @ values 
        return context_vec
    
torch.manual_seed(123) 
sa_v1 = SelfAttention_v1(d_in, d_out) 
print(sa_v1(inputs))

tensor([[0.2996, 0.8053],
        [0.3061, 0.8210],
        [0.3058, 0.8203],
        [0.2948, 0.7939],
        [0.2927, 0.7891],
        [0.2990, 0.8040]], grad_fn=<MmBackward0>)


In [25]:
# using nn.Linear

class SelfAttention_v2(nn.Module):    
    def __init__(self, d_in, d_out, qkv_bias=False):
        super().__init__() 
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias) 
        self.W_key   = nn.Linear(d_in, d_out, bias=qkv_bias) 
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)

    def forward(self, x): 
        keys = self.W_key(x) 
        queries = self.W_query(x)
        values = self.W_value(x)
        attn_scores = queries @ keys.T # omega 
        attn_weights = torch.softmax( attn_scores / keys.shape[-1]**0.5, dim=-1 ) 
        context_vec = attn_weights @ values 
        return context_vec
    
torch.manual_seed(789) 
sa_v2 = SelfAttention_v2(d_in, d_out) 
print(sa_v2(inputs))

tensor([[-0.0739,  0.0713],
        [-0.0748,  0.0703],
        [-0.0749,  0.0702],
        [-0.0760,  0.0685],
        [-0.0763,  0.0679],
        [-0.0754,  0.0693]], grad_fn=<MmBackward0>)
